In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import os
os.environ['KERAS_BACKEND'] = 'theano'
import keras as ks

Using TensorFlow backend.


In [0]:
import pickle

In [0]:
target1_token_index=pickle.load(open("dictionary.pkl","rb"))

In [0]:
target1_token_index

In [0]:
morpheme_list=pickle.load(open("morpheme_new_list.pkl","rb"))

In [11]:
lines= pd.read_table('mar1.txt', names=['eng', 'fre'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [0]:
len(lines)

2000

In [0]:
#lines=lines[0:11725]

In [0]:
lines.mar

In [0]:
lines.shape

(2000, 2)

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fre=lines.fre.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.fre=lines.fre.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fre=lines.fre.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fre = lines.fre.apply(lambda x: re.sub("[1230456789]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.fre=lines.fre.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.fre=lines.fre.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.fre = lines.fre.apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_french_words=set()
for mar in lines.fre:
    for word in mar.split():
        print(word)
        if word not in all_french_words:
            all_french_words.add(word)

In [20]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

4

In [0]:
max_length_src=10


In [0]:
new_word_morpheme_list=pickle.load(open("morpheme_new_list.pkl","rb"))

In [22]:
len(new_word_morpheme_list)

5458

In [23]:
# Max Length of target sequence
lenght_list=[]
for l in lines.fre:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

10

In [0]:
all_marathi_words

In [24]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(morpheme_list))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(morpheme_list)
#num_morph_dec_tokens=len(moepheme_list)
num_encoder_tokens, num_decoder_tokens

(637, 5458)

In [0]:
len(all_eng_words)

637

In [0]:
len(new_word_morpheme_list)

5458

In [25]:
num_decoder_tokens += 3 # For zero padding
num_decoder_tokens

5461

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
#target1_token_index = dict([(word, i+1) for i, word in enumerate(morph_words)])

In [0]:
len(target2_token_index)

689

In [0]:
target_token_index ['START_']=5459

In [0]:
target_token_index  ['_END']=5460

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [33]:
lines = shuffle(lines)
lines.head(10)

,eng,fre
684,of course,START_ mais ouais _END
320,leave us,START_ laissenous _END
231,hi guys,START_ salut les mecs _END
873,come quick,START_ jouis vite _END
1208,its wrong,START_ cest faux _END
1955,i will help,START_ jaiderai _END
915,he gave in,START_ il céda _END
1184,its a fad,START_ cest une passade _END
183,we lost,START_ nous avons été battues _END
1774,i got dizzy,START_ jai été prise de vertiges _END


In [35]:
# Train - Test Split
X, y = lines.eng, lines.fre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((1800,), (200,))

In [0]:
y_train.shape

(1800,)

In [0]:
import pickle

In [0]:
pickle.dump(X_train,open("X_test.pkl","wb"))

In [0]:
pickle.dump(X_train,open("X_train.pkl","wb"))

In [0]:
target1_token_index['START_\n']='START_'
target1_token_index['_END\n']='_END'

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 16):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        word1=word+"\n"
                        word1=target1_token_index[word1]
                        decoder_input_data[i, t] = target_token_index[word1] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        word1=word+"\n"
                        word1=target1_token_index[word1]
                        decoder_target_data[i, t - 1, target_token_index[word1]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [43]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

W0705 10:41:49.894991 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0705 10:41:49.919199 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0705 10:41:49.925367 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0705 10:41:50.184245 140584343533440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruct

In [0]:
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
#decoder_morph_inputs = Input(shape=(None,))
#merge_one = concatenate([decoder_inputs, decoder_morph_inputs])
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
#dec_morph_emb_layer = Embedding(num_morph_dec_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
#dec_morph_emb= dec_morph_emb_layer(decoder_morph_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
#dec_new_emb=merge([dec_emb,dec_morph_emb],mode='sum')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
#decoder_morph_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
#decoder_morph_outputs,_, _  = decoder_morph_lstm(dec_morph_emb,
                                     #initial_state=encoder_states)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
#decoder_morph_dense = Dense(num_morph_dec_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
#decoder_morph_outputs=decoder_dense(decoder_morph_outputs)
#tot_decoder_outputs = concatenate([decoder_outputs, decoder_morph_outputs])
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 5

In [49]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

W0705 10:42:55.021048 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0705 10:42:55.057593 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [50]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

W0705 10:42:59.693342 140584343533440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
14/14 [==============================] - 4s 263ms/step - loss: 8.5060 - acc: 0.2472 - val_loss: 8.1706 - val_acc: 0.2628
Epoch 2/5
14/14 [==============================] - 1s 80ms/step - loss: 7.2542 - acc: 0.2664 - val_loss: 6.2867 - val_acc: 0.2687
Epoch 3/5
14/14 [==============================] - 1s 78ms/step - loss: 5.6874 - acc: 0.2668 - val_loss: 5.3219 - val_acc: 0.2628
Epoch 4/5
14/14 [==============================] - 1s 77ms/step - loss: 4.9904 - acc: 0.2671 - val_loss: 4.9373 - val_acc: 0.2687
Epoch 5/5
14/14 [==============================] - 1s 77ms/step - loss: 4.6795 - acc: 0.2668 - val_loss: 4.6809 - val_acc: 0.2628
